<a href="https://colab.research.google.com/github/etuckerman/LLaMA-3.1-QA-Finetune/blob/main/LLaMA_3_1_QA_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).

# import dataset (16 qna's)

In [2]:
#pip dump
!pip install transformers datasets
!pip install scikit-learn
!pip install bitsandbytes
!pip install peft==0.4.0
# !pip install transformers==4.31.0
# !pip install accelerate==0.21.0
!pip install --upgrade transformers
!pip install --upgrade accelerate

In [3]:
import pandas as pd

# Define the data
data = {
    'Question': [
        "Does the Jayco Eagle have a queen size bed?",
        "What is the MSRP of the Jayco Eagle?",
        "How many people can sleep in the Jayco Eagle?",
        "Does the Jayco Eagle have a shower?",
        "Does the Keystone Cougar have a queen size bed?",
        "What is the MSRP of the Keystone Cougar?",
        "How many people can sleep in the Keystone Cougar?",
        "Does the Keystone Cougar have a shower?",
        "Does the Coachmen Clipper Sport have a queen size bed?",
        "What is the MSRP of the Coachmen Clipper Sport?",
        "How many people can sleep in the Coachmen Clipper Sport?",
        "Does the Coachmen Clipper Sport have a shower?",
        "Does the Fleetwood Fiesta have a queen size bed?",
        "What is the MSRP of the Fleetwood Fiesta?",
        "How many people can sleep in the Fleetwood Fiesta?",
        "Does the Fleetwood Fiesta have a shower?"
    ],
    'Answer': [
        "Yes, the Jayco Eagle has 1 queen size bed.",
        "The MSRP of the Jayco Eagle is $32,186.",
        "The Jayco Eagle can sleep up to 9 people.",
        "Yes, the Jayco Eagle comes with a standard shower.",
        "Yes, the Keystone Cougar has 1 queen size bed.",
        "The MSRP of the Keystone Cougar is $28,603.",
        "The Keystone Cougar can sleep up to 6 people.",
        "Yes, the Keystone Cougar comes with a standard shower.",
        "No, the Coachmen Clipper Sport does not have a queen size bed.",
        "The MSRP of the Coachmen Clipper Sport is $6,083.",
        "The Coachmen Clipper Sport can sleep up to 6 people.",
        "No, the Coachmen Clipper Sport does not have a shower available.",
        "No, the Fleetwood Fiesta does not have a queen size bed.",
        "The MSRP of the Fleetwood Fiesta is $7,231.",
        "The Fleetwood Fiesta can sleep up to 6 people.",
        "No, the Fleetwood Fiesta does not have a shower available."
    ]
}

In [4]:
import numpy as np

#save data as dataframe
df = pd.DataFrame(data)

#create a random split and assign labels
labels = np.random.choice(['train', 'test', 'validation'], size=len(df), p=[0.7, 0.15, 0.15])
df['split'] = labels

In [5]:
#save data as csv
df.to_csv('qa_examples.csv', index=False)


## Load dataset

In [6]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='qa_examples.csv', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['Question', 'Answer', 'split'],
    num_rows: 16
})

# Fine-Tuning the LLaMA 3.1 Model

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
#model name
#model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" #temp model

In [10]:
#load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


In [12]:
# #load model with low memory footprint (using model checkpoint)
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")


In [13]:
# Load the base model
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.08480376642881861


In [14]:
# from bitsandbytes.optim import Adam8bit

# #not enough cuda ram, use bitsandbytes to quantize
# # optimizer for 4-bit precision
# optimizer = Adam8bit(model.parameters(), lr=3e-5)


In [15]:
#check if  tokenizer has a padding token
if tokenizer.pad_token is None:
    #if not, set one
    #use end of sentence token as pad_token or add a new pad token
    tokenizer.pad_token = tokenizer.eos_token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        tokenizer.pad_token = '[PAD]'

In [16]:
def preprocess_data(examples):
    inputs = examples['Question']
    targets = examples['Answer']
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=256)

    #tokenize labels (answers)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, padding="max_length", truncation=True, max_length=256)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [17]:
#convert to Hugging Face Dataset
hf_dataset = dataset.from_pandas(df)

In [18]:
#tokenize the dataset using AutoTokenizer
tokenized_dataset = hf_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [19]:
print(tokenized_dataset)

Dataset({
    features: ['Question', 'Answer', 'split', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 16
})


In [20]:
train_dataset = tokenized_dataset.filter(lambda example: example['split'] == 'train')
validation_dataset = tokenized_dataset.filter(lambda example: example['split'] == 'validation')


Filter:   0%|          | 0/16 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16 [00:00<?, ? examples/s]

In [21]:
print(train_dataset)

Dataset({
    features: ['Question', 'Answer', 'split', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 15
})


In [22]:
print(validation_dataset)

Dataset({
    features: ['Question', 'Answer', 'split', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})


In [23]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,  # Higher learning rate for LoRA
    per_device_train_batch_size=8,  # Reduced batch size
    per_device_eval_batch_size=8,  # Reduced batch size
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False, # Set to False if you don't want to push to Hugging Face Hub
    logging_dir="./logs",
    logging_steps=10,
    optim="paged_adamw_8bit",  # Use 8-bit optimizer for efficiency
    save_total_limit=2,
    load_best_model_at_end=True,
)

In [24]:
from transformers import Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding( #auto pad input seq to same length within a batch
    tokenizer=tokenizer  # ensures padding is done based on the tokenizer's requirements
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    # optimizers=(optimizer, None)  #optimizer for 8-bit precision

)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [25]:
!shell
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
# start training
trainer.train()


/bin/bash: line 1: shell: command not found


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 10.81 MiB is free. Process 58825 has 39.54 GiB memory in use. Of the allocated memory 39.02 GiB is allocated by PyTorch, and 34.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)